In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import optuna
import plotly.express as px

# Load and prepare data
DATA_PATH = 'german_credit_data_cleaned.csv'
df = pd.read_csv(DATA_PATH)
y = df['Risk_good'].values
X_full = df.drop(columns=['Risk_good'])
sensitive = df['Sex_male'].values



c:\Users\ayush\OneDrive - iiit-b\CollegeAcads\Sem6\MOML\Project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Define the custom objective function factory
def make_custom_obj(sens_array, alpha=0.5):
    def custom_obj(preds, dmatrix):
        labels = dmatrix.get_label()
        preds_prob = 1.0 / (1.0 + np.exp(-preds))
        grad_log = preds_prob - labels
        hess_log = preds_prob * (1.0 - preds_prob)
        mask_p = (sens_array == 1)
        mask_u = (sens_array == 0)
        n_p, n_u = mask_p.sum(), mask_u.sum()
        mean_p = preds_prob[mask_p].mean() if n_p > 0 else 0
        mean_u = preds_prob[mask_u].mean() if n_u > 0 else 0
        diff = mean_p - mean_u
        sign = np.sign(diff)
        sigmoid_deriv = preds_prob * (1.0 - preds_prob)
        grad_fair = np.zeros_like(preds)
        grad_fair[mask_p] = sign * (1.0 / n_p) * sigmoid_deriv[mask_p] if n_p > 0 else 0
        grad_fair[mask_u] = -sign * (1.0 / n_u) * sigmoid_deriv[mask_u] if n_u > 0 else 0
        hess_fair = np.zeros_like(preds)
        grad = alpha * grad_log + (1 - alpha) * grad_fair
        hess = alpha * hess_log + (1 - alpha) * hess_fair
        return grad, hess
    return custom_obj

# Define the objective function for Optuna

def create_objective_function(alpha):
    def objective(trial):
        # Suggest hyperparameters
        # alpha = trial.suggest_float('alpha', 0.0, 1.0)
        max_depth = trial.suggest_int('max_depth', 3, 7)
        eta = trial.suggest_float('eta', 0.01, 0.3)
        subsample = trial.suggest_float('subsample', 0.6, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)

        # Initialize cross-validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        acc_scores = []
        dpd_scores = []

        # Perform 3-fold cross-validation
        for train_idx, val_idx in kf.split(X_full):
            # Split data
            X_train = X_full.iloc[train_idx].drop(columns=['Sex_male']).values
            y_train = y[train_idx]
            sens_train = sensitive[train_idx]
            X_val = X_full.iloc[val_idx].drop(columns=['Sex_male']).values
            y_val = y[val_idx]
            sens_val = sensitive[val_idx]

            # Create custom objective
            train_obj = make_custom_obj(sens_train, alpha=alpha)

            # Set up DMatrix
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)

            # Train the model
            params = {
                'max_depth': max_depth,
                'eta': eta,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree,
                'verbosity': 0
            }
            bst = xgb.train(
                params,
                dtrain,
                num_boost_round=100,
                obj=train_obj,
                evals=[(dtrain, 'train'), (dval, 'val')],
                early_stopping_rounds=10,
                verbose_eval=False
            )

            # Predict on validation set
            pred_prob = bst.predict(dval)
            pred = (pred_prob > 0.5).astype(int)

            # Compute accuracy and DPD
            acc = accuracy_score(y_val, pred)
            dpd_val = abs(pred[sens_val == 1].mean() - pred[sens_val == 0].mean())
            acc_scores.append(acc)
            dpd_scores.append(dpd_val)

        # Compute mean metrics across folds
        mean_acc = np.mean(acc_scores)
        mean_dpd = np.mean(dpd_scores)

        # Store metrics for analysis
        trial.set_user_attr('accuracy', mean_acc)
        trial.set_user_attr('dpd', mean_dpd)
        trial.set_user_attr('params', {
            'alpha': alpha,
            'max_depth': max_depth,
            'eta': eta,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree
        })

        return mean_acc, mean_dpd
    return objective


In [39]:

# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient
#adaptive weights based on distance

results = []
count_alpha = 100
trials_per_alpha = 10
for i in range(count_alpha+1):
    alpha = i/count_alpha
    study = optuna.create_study(directions=['maximize', 'minimize'])
    objective = create_objective_function(alpha=alpha)  # Example alpha value
    study.optimize(objective, n_trials=trials_per_alpha)
    print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })

acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-15 15:59:28,928] A new study created in memory with name: no-name-29dc98ff-ffe0-47f8-8e54-12bc9b492069
[I 2025-05-15 15:59:29,030] Trial 0 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 6, 'eta': 0.16395389636146293, 'subsample': 0.8000194327801043, 'colsample_bytree': 0.6748470800330576}.
[I 2025-05-15 15:59:29,123] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 4, 'eta': 0.01191635966343524, 'subsample': 0.9575531224176509, 'colsample_bytree': 0.9133666824219422}.
[I 2025-05-15 15:59:29,215] Trial 2 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 4, 'eta': 0.21810271410930646, 'subsample': 0.6048986751774701, 'colsample_bytree': 0.9335176744047675}.
[I 2025-05-15 15:59:29,299] Trial 3 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.25238092029771686, 'subsample': 0.8649728036602894, 'colsample_bytree': 0.9277386344843667}.
[I 2025-05

Completed study for alpha=0.0


[I 2025-05-15 15:59:30,025] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 4, 'eta': 0.05437000691947621, 'subsample': 0.7651516278034398, 'colsample_bytree': 0.9504073956911484}.
[I 2025-05-15 15:59:30,245] Trial 2 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 3, 'eta': 0.15980571778136424, 'subsample': 0.8406247341941625, 'colsample_bytree': 0.9995251148363173}.
[I 2025-05-15 15:59:30,364] Trial 3 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 5, 'eta': 0.2928385377644047, 'subsample': 0.7282232095404597, 'colsample_bytree': 0.790471506597261}.
[I 2025-05-15 15:59:30,469] Trial 4 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.17379860191004665, 'subsample': 0.7775145221316209, 'colsample_bytree': 0.692514616496732}.
[I 2025-05-15 15:59:30,578] Trial 5 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 4, 'eta': 0.0

Completed study for alpha=0.01


[I 2025-05-15 15:59:31,205] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 4, 'eta': 0.09531837603665186, 'subsample': 0.642385317398397, 'colsample_bytree': 0.6949188032100334}.
[I 2025-05-15 15:59:31,314] Trial 2 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 6, 'eta': 0.17856409320804587, 'subsample': 0.8068202785048436, 'colsample_bytree': 0.820388693296456}.
[I 2025-05-15 15:59:31,464] Trial 3 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 3, 'eta': 0.043746391631996125, 'subsample': 0.9178406630545798, 'colsample_bytree': 0.6799722431440286}.
[I 2025-05-15 15:59:31,598] Trial 4 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 4, 'eta': 0.09274541631596356, 'subsample': 0.9991503525088102, 'colsample_bytree': 0.816586728094092}.
[I 2025-05-15 15:59:31,897] Trial 5 finished with values: [0.66001599103031

...

Completed study for alpha=0.98


[I 2025-05-15 16:02:03,813] Trial 1 finished with values: [0.6678675256484485, 0.03833130745395059] and parameters: {'max_depth': 3, 'eta': 0.2440000162855522, 'subsample': 0.9746964700168641, 'colsample_bytree': 0.8824161200413989}.
[I 2025-05-15 16:02:03,943] Trial 2 finished with values: [0.6857134542157196, 0.08274115871786776] and parameters: {'max_depth': 4, 'eta': 0.2637817934950271, 'subsample': 0.6834888074468948, 'colsample_bytree': 0.7942067934719065}.
[I 2025-05-15 16:02:04,088] Trial 3 finished with values: [0.6864272300900951, 0.07442476296409668] and parameters: {'max_depth': 5, 'eta': 0.1270819680730535, 'subsample': 0.7230649833765055, 'colsample_bytree': 0.9026664272694651}.
[I 2025-05-15 16:02:04,246] Trial 4 finished with values: [0.6971353999136117, 0.06243665285241878] and parameters: {'max_depth': 7, 'eta': 0.14917082073696242, 'subsample': 0.9435805293025702, 'colsample_bytree': 0.6115318426105149}.
[I 2025-05-15 16:02:04,387] Trial 5 finished with values: [0.72

Completed study for alpha=0.99


[I 2025-05-15 16:02:05,582] Trial 1 finished with values: [0.6935634571259645, 0.08166426055103489] and parameters: {'max_depth': 7, 'eta': 0.011681836562825723, 'subsample': 0.947756995249263, 'colsample_bytree': 0.8498739788871004}.
[I 2025-05-15 16:02:05,736] Trial 2 finished with values: [0.7035746998618398, 0.08811416806669899] and parameters: {'max_depth': 5, 'eta': 0.29992722957016227, 'subsample': 0.6743553630299758, 'colsample_bytree': 0.7342161105820793}.
[I 2025-05-15 16:02:05,848] Trial 3 finished with values: [0.6664384421918127, 0.03461200648039473] and parameters: {'max_depth': 3, 'eta': 0.2172056569741549, 'subsample': 0.8923841631840115, 'colsample_bytree': 0.6601596667791629}.
[I 2025-05-15 16:02:05,992] Trial 4 finished with values: [0.6921359053772136, 0.06616182238167867] and parameters: {'max_depth': 5, 'eta': 0.14235611798036324, 'subsample': 0.9262807154160297, 'colsample_bytree': 0.8947199568112114}.
[I 2025-05-15 16:02:06,122] Trial 5 finished with values: [0.

Completed study for alpha=1.0
Pareto-optimal solutions:
Params: {'alpha': 0.0, 'max_depth': 6, 'eta': 0.16395389636146293, 'subsample': 0.8000194327801043, 'colsample_bytree': 0.6748470800330576}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.01, 'max_depth': 4, 'eta': 0.24688563768301078, 'subsample': 0.6086094519850038, 'colsample_bytree': 0.8988827726356343}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.02, 'max_depth': 4, 'eta': 0.09531837603665186, 'subsample': 0.642385317398397, 'colsample_bytree': 0.6949188032100334}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.03, 'max_depth': 4, 'eta': 0.09727112470519857, 'subsample': 0.6788574650935559, 'colsample_bytree': 0.9608639352269196}, Accuracy: 0.6707, DPD: 0.0130
Params: {'alpha': 0.04, 'max_depth': 4, 'eta': 0.033693738058008856, 'subsample': 0.6035248461068744, 'colsample_bytree': 0.6947171582505717}, Accuracy: 0.6707, DPD: 0.0130
Params: {'alpha': 0.18, 'max_depth': 7, 'eta': 0.1985942254147221, 'subsample': 0.774

In [40]:
# Create interactive Plotly plot
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()

In [30]:

# Create and run Optuna study for multi-objective optimization
study = optuna.create_study(directions=['maximize', 'minimize'])
objective = create_objective_function(alpha=np.random.uniform(0,1))  # Example alpha value
study.optimize(objective, n_trials=100)

# Collect Pareto front results
results = []
for trial in study.best_trials:  # Best trials are the Pareto-optimal ones
    results.append({
        'accuracy': trial.values[0],
        'dpd': trial.values[1],
        'alpha': trial.user_attrs['params']['alpha'],
        'params': trial.user_attrs['params']
    })

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-15 04:47:08,768] A new study created in memory with name: no-name-81c9934b-2059-41fa-9a1d-ffa5aeeee5de
[I 2025-05-15 04:47:08,897] Trial 0 finished with values: [0.6792971298857652, 0.08215217241285708] and parameters: {'max_depth': 5, 'eta': 0.14045260605383383, 'subsample': 0.6059859634768344, 'colsample_bytree': 0.9142953795430282}.
[I 2025-05-15 04:47:08,995] Trial 1 finished with values: [0.6600175227382036, 0.015081357718660824] and parameters: {'max_depth': 3, 'eta': 0.05033127668007191, 'subsample': 0.7887078508400488, 'colsample_bytree': 0.8895195187557907}.
[I 2025-05-15 04:47:09,066] Trial 2 finished with values: [0.6764435580961484, 0.04444123543631445] and parameters: {'max_depth': 3, 'eta': 0.2939283612519244, 'subsample': 0.8063484386283546, 'colsample_bytree': 0.951394948420871}.
[I 2025-05-15 04:47:09,139] Trial 3 finished with values: [0.667858335401139, 0.05062834891783413] and parameters: {'max_depth': 4, 'eta': 0.11946224856843993, 'subsample': 0.8938040

Pareto-optimal solutions:
Params: {'alpha': 0.41383153948093454, 'max_depth': 7, 'eta': 0.2804130195047165, 'subsample': 0.8838936125401613, 'colsample_bytree': 0.7188154735379857}, Accuracy: 0.7021, DPD: 0.0689
Params: {'alpha': 0.41383153948093454, 'max_depth': 5, 'eta': 0.2789019660094398, 'subsample': 0.8824040017757933, 'colsample_bytree': 0.6075094742604826}, Accuracy: 0.6914, DPD: 0.0536
Params: {'alpha': 0.41383153948093454, 'max_depth': 4, 'eta': 0.2714574989051898, 'subsample': 0.9284415700920252, 'colsample_bytree': 0.8076193158434031}, Accuracy: 0.6886, DPD: 0.0439
Params: {'alpha': 0.41383153948093454, 'max_depth': 6, 'eta': 0.20020253145881808, 'subsample': 0.8267274788350784, 'colsample_bytree': 0.6612102025456387}, Accuracy: 0.6843, DPD: 0.0384
Params: {'alpha': 0.41383153948093454, 'max_depth': 5, 'eta': 0.2939283612519244, 'subsample': 0.6818951506364179, 'colsample_bytree': 0.6589253591052271}, Accuracy: 0.6922, DPD: 0.0571
Params: {'alpha': 0.41383153948093454, 'max

In [61]:
# Create interactive Plotly plot
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()

In [60]:

# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient

def normalise(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)
#adaptive weights based on distance

results = []
trials_per_alpha = 50
study = optuna.create_study(directions=['maximize', 'minimize'])
objective = create_objective_function(alpha=0)  # Example alpha value
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=0")
max_val = 0
for trial in study.best_trials:
    val = test_loss(alpha,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 0,
    'params': best_trial.user_attrs['params']
    })
study = optuna.create_study(directions=['maximize', 'minimize'])
objective = create_objective_function(alpha=1)  # Example alpha value
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=1")
max_val = 0
for trial in study.best_trials:
    val = test_loss(alpha,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 1,
    'params': best_trial.user_attrs['params']
})
print(results)

max_acc = max([res['accuracy'] for res in results])
max_dpd = max([res['dpd'] for res in results])
min_acc = min([res['accuracy'] for res in results])
min_dpd = min([res['dpd'] for res in results])
count_alpha = 20



for i in range(count_alpha):
    print(i)
    
    results.sort(key=lambda x: x['alpha'])
    max_dist = 0
    for i in range(len(results)-1):
        acc_dist = (normalise(results[i]['accuracy'],min_acc,max_acc)-normalise(results[i+1]['accuracy'],min_acc,max_acc))**2
        dpd_dist = (normalise(results[i]['dpd'],min_dpd,max_dpd)-normalise(results[i+1]['dpd'],min_dpd,max_dpd))**2

        dist = np.sqrt(acc_dist + dpd_dist)
        if dist > max_dist:
            max_dist = dist
            alpha = (results[i]['alpha']+results[i+1]['alpha'])/2
    study = optuna.create_study(directions=['maximize', 'minimize'])
    objective = create_objective_function(alpha=alpha)  # Example alpha value
    study.optimize(objective, n_trials=trials_per_alpha)
    print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
        
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })
    max_acc = max(max_acc, best_trial.values[0])
    max_dpd = max(max_dpd, best_trial.values[1])
    min_acc = min(min_acc, best_trial.values[0])
    min_dpd = min(min_dpd, best_trial.values[1])
acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-15 16:49:25,765] A new study created in memory with name: no-name-1cc00296-c997-4db7-a120-3a57cb82af32
[I 2025-05-15 16:49:25,854] Trial 0 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.22571785870579533, 'subsample': 0.7549245617609082, 'colsample_bytree': 0.8564068669101881}.
[I 2025-05-15 16:49:25,973] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 3, 'eta': 0.012426119273396987, 'subsample': 0.7144774828630672, 'colsample_bytree': 0.6998200934807096}.
[I 2025-05-15 16:49:26,049] Trial 2 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 3, 'eta': 0.029721369826485183, 'subsample': 0.9803276534509726, 'colsample_bytree': 0.6863272337167018}.
[I 2025-05-15 16:49:26,135] Trial 3 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 5, 'eta': 0.0633415980403137, 'subsample': 0.8564704235543256, 'colsample_bytree': 0.8407084973375782}.
[I 2025-0

Completed study for alpha=0


[I 2025-05-15 16:49:30,842] Trial 1 finished with values: [0.6871486645038951, 0.057546422151730975] and parameters: {'max_depth': 4, 'eta': 0.1632886394996874, 'subsample': 0.8066345957273467, 'colsample_bytree': 0.6526140059169697}.
[I 2025-05-15 16:49:31,083] Trial 2 finished with values: [0.6792910030542255, 0.055470606173256] and parameters: {'max_depth': 4, 'eta': 0.19975771044103077, 'subsample': 0.913869761323053, 'colsample_bytree': 0.9706239837737644}.
[I 2025-05-15 16:49:31,320] Trial 3 finished with values: [0.6707210974380654, 0.05923291155769873] and parameters: {'max_depth': 4, 'eta': 0.010270547453499273, 'subsample': 0.8663655280495077, 'colsample_bytree': 0.6502023763678864}.
[I 2025-05-15 16:49:31,406] Trial 4 finished with values: [0.6778634513054747, 0.07524724202665488] and parameters: {'max_depth': 4, 'eta': 0.19026604105780212, 'subsample': 0.6331450465982246, 'colsample_bytree': 0.7737646145843996}.
[I 2025-05-15 16:49:31,505] Trial 5 finished with values: [0.6

Completed study for alpha=1
[{'accuracy': 0.5000061268315397, 'dpd': 0.0, 'alpha': 0, 'params': {'alpha': 0, 'max_depth': 7, 'eta': 0.22571785870579533, 'subsample': 0.7549245617609082, 'colsample_bytree': 0.8564068669101881}}, {'accuracy': 0.6614420110711846, 'dpd': 0.022685717044366272, 'alpha': 1, 'params': {'alpha': 1, 'max_depth': 3, 'eta': 0.07181610823749664, 'subsample': 0.9993702966207471, 'colsample_bytree': 0.9283806506077897}}]
0


[I 2025-05-15 16:49:37,785] Trial 1 finished with values: [0.6607251717810393, 0.04979021432358924] and parameters: {'max_depth': 6, 'eta': 0.036893774999760355, 'subsample': 0.869207702278322, 'colsample_bytree': 0.6461593280240302}.
[I 2025-05-15 16:49:37,902] Trial 2 finished with values: [0.6635879338179658, 0.028020161188967174] and parameters: {'max_depth': 3, 'eta': 0.1474832758512428, 'subsample': 0.7294422138604334, 'colsample_bytree': 0.9639241388690059}.
[I 2025-05-15 16:49:38,146] Trial 3 finished with values: [0.6728593616454219, 0.0661348397964356] and parameters: {'max_depth': 7, 'eta': 0.013842942804147745, 'subsample': 0.6576352065259878, 'colsample_bytree': 0.86483581713232}.
[I 2025-05-15 16:49:38,383] Trial 4 finished with values: [0.6735731375197972, 0.04312784221716868] and parameters: {'max_depth': 6, 'eta': 0.02469295552313607, 'subsample': 0.796429281164034, 'colsample_bytree': 0.9401413783999186}.
[I 2025-05-15 16:49:38,502] Trial 5 finished with values: [0.67

...

Completed study for alpha=0.021953582763671875
18


[I 2025-05-15 16:51:33,502] Trial 1 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 3, 'eta': 0.25526308788862845, 'subsample': 0.8995033041503491, 'colsample_bytree': 0.9628472658496581}.
[I 2025-05-15 16:51:33,614] Trial 2 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 7, 'eta': 0.18747919735018814, 'subsample': 0.9218334233481711, 'colsample_bytree': 0.972625596300286}.
[I 2025-05-15 16:51:33,790] Trial 3 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 7, 'eta': 0.017567406703553137, 'subsample': 0.6699198615188144, 'colsample_bytree': 0.7963994365330949}.
[I 2025-05-15 16:51:33,919] Trial 4 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 6, 'eta': 0.05031460650431966, 'subsample': 0.9655953780013955, 'colsample_bytree': 0.9172729319726947}.
[I 2025-05-15 16:51:34,034] Trial 5 finished with values

Completed study for alpha=0.021955490112304688
19


[I 2025-05-15 16:51:39,618] Trial 1 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 5, 'eta': 0.1615306409529793, 'subsample': 0.9861369070972635, 'colsample_bytree': 0.8428728967737957}.
[I 2025-05-15 16:51:39,728] Trial 2 finished with values: [0.6607297669046941, 0.025165936256544008] and parameters: {'max_depth': 6, 'eta': 0.21185315801615404, 'subsample': 0.6580581699186204, 'colsample_bytree': 0.7598215670205795}.
[I 2025-05-15 16:51:39,835] Trial 3 finished with values: [0.5922057512567663, 0.011420900198076733] and parameters: {'max_depth': 7, 'eta': 0.17862026002616652, 'subsample': 0.6087143319361482, 'colsample_bytree': 0.9232843240126506}.
[I 2025-05-15 16:51:39,954] Trial 4 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 3, 'eta': 0.059185425361644714, 'subsample': 0.8852612468458297, 'colsample_bytree': 0.9238479930822426}.
[I 2025-05-15 16:51:40,057] Trial 5 finished with values

Completed study for alpha=0.021956443786621094
Pareto-optimal solutions:
Params: {'alpha': 0, 'max_depth': 7, 'eta': 0.22571785870579533, 'subsample': 0.7549245617609082, 'colsample_bytree': 0.8564068669101881}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.015625, 'max_depth': 7, 'eta': 0.14688225355269602, 'subsample': 0.8561928535368462, 'colsample_bytree': 0.7885165329373263}, Accuracy: 0.5264, DPD: 0.0001
Params: {'alpha': 0.01953125, 'max_depth': 7, 'eta': 0.2488700352293079, 'subsample': 0.6385395752585328, 'colsample_bytree': 0.8340665174347298}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.021484375, 'max_depth': 3, 'eta': 0.0472164139827578, 'subsample': 0.6138702636232714, 'colsample_bytree': 0.7448819581306698}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.021728515625, 'max_depth': 5, 'eta': 0.018658643342671855, 'subsample': 0.6072812903877419, 'colsample_bytree': 0.9270656391411835}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.0218505859375, 'max_dept